***'Syntax natürlicher Sprachen',*** **Wintersemester 2022/23**

--- 
# Klausur: Syntax natürlicher Sprachen *(14.02.2023)*

- **Bearbeitungszeitraum: 10:15-11:45**
- **Abgabe in Moodle bis 12:00**


## Übersicht

- [Hinweise zur Bearbeitung](#Hinweise-zur-Bearbeitung)
- [Laden von Paketen](#Laden-von-Paketen)


1. [Ambiguität in Konstituentengrammatik](#1.-Ambiguität-in-Konstituentengrammatik)
2. [Ambiguität in Dependenzgrammatik](#2.-Ambiguität-in-Dependenzgrammatik)
3. [Konstituenten- und Adjunkttests](#3.-Konstituenten--und-Adjunkttests)
4. [CFG Analysen](#4.-CFG-Analysen)
5. [Dependenzanalyse](#5.-Dependenzanalyse)
6. [Feature-based-Grammar](#6.-Feature-based-Grammar)
7. [Slash-Kategorien](#7.-Slash-Kategorien)
8. [Syntaxregeln komplexer Sätze](#8.-Syntaxregeln-komplexer-Sätze)
9. [Parsing-Algorithmen und Rekursionstypen](#9.-Parsing-Algorithmen-und-Rekursionstypen)
10. [Unifikationsparsing](#10.-Unifikationsparsing-und-getypte-Merkmalstrukturen)
11. [Statistisches Parsing](#11.-Statistisches-Parsing)
12. [Datengestützte Syntaxanalyse](#12.-Datengestützte-Syntaxanalyse)
13. [Chunk-Analysen](#13.-Chunk-Analysen)





---

## Hinweise zur Bearbeitung

### Falls noch nicht geschehen, benennen Sie bitte zunächst die Datei der Klausur-Angabe nach folgendem Schema um: 

 #### `Nachname_Vorname_Matrikelnummer.ipynb`
 

### Geben Sie nun in folgender Codezelle die letzte Ziffer Ihrer Matrikelnummer ein (`last_number`) und führen Sie die Codezelle aus, um Ihre Gruppennummer zu berechnen:

In [ ]:
last_number = 

group_number = int((((last_number*3-2)%10)/4.0))+1
print("Ihre Gruppennummer: "+str(group_number))

---
## *WICHTIG: Bearbeiten Sie in den Aufgaben, in denen explizit darauf hingewiesen wird, nur die Aufgabenstellung, die Ihnen gemäß Ihrer Gruppennummer zugeordnet wird!*


--- 

## Hinweise zum Ausfüllen der Codezellen

### Wenn gegeben, führen Sie zunächst die mit `#RUN` markierten Codezellen zu Beginn einer Aufgabe aus (dies ist für eine erfolgreiche Bearbeitung der Aufgabe notwendig).

### Verändern Sie nur die `#TO_DO`-Codezellen (nur gemäß der Angabe in der jeweiligen Aufgabe)!

### Führen Sie `#TO_DO`-Codezellen nach Bearbeitung aus, um das Output ihrer Lösung zu generieren (dieses muss als Teil Ihrer Lösung mit abgespeichert werden); bei aufeinander aufbauenden Aufgaben (`a)`, `b)`usw.) ist zudem  notwendig, dass Sie Ihre Lösung aus der vorangehenden Teilaufgabe ausführen, damit diese in der folgenden zur Verfügung steht.

### Angegebene Inhalte (Grammatikregeln usw.) dürfen nicht auskommentiert oder gelöscht werden, außer dies wird explizit anders erwähnt!


### WICHTIG: Setzen Sie den Status des Notebooks ggf. auf `Trusted`, damit alle angegebenen Outputs korrekt angezeigt werden.

---

## Hinweis zur Bewertung

### *Jede korrekt gelöste Teilaufgabe (`#TO_DO`-Codezelle) wird mit 2 Punkten bewertet*

---

[zurück zur Übersicht](#Übersicht)


---

## Laden von Paketen

### Führen Sie zu Beginn folgende Codezelle aus.

### Das erfolgreiche Ausführen dieser Codezelle ist Voraussetzung für die Bearbeitung der folgenden Aufgaben.  

In [1]:
#RUN (Führen Sie diese Code-Zelle aus:)
import nltk
from nltk.tree import Tree
from nltk import FeatStruct
import itertools


from spacy import displacy

def transform_nr_conll(sent_nr):
    sent_list = []
    for line in list(filter(None, sent_nr.split("\n"))):
        line_list = line.split()
        line_list.pop(0)
        line_list.insert(1,"_")
        sent_list.append(" ".join([i for i in line_list[0:]]))

    return "\n".join([i for i in sent_list[0:]])



from nltk import DependencyGraph
from itertools import chain

def _tree_labeled(self, i):
        node = self.get_by_address(i)
        word = node["word"]
        rel = node["rel"]        
        deps = sorted(chain.from_iterable(node["deps"].values()))

        if deps:
            return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])
        else:
            return word+'('+rel+')'
        
def tree_labeled(self):
        node = self.root

        word = node["word"]
        rel = node["rel"]
        deps = sorted(chain.from_iterable(node["deps"].values()))
        return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])

DependencyGraph._tree_labeled = _tree_labeled
DependencyGraph.tree_labeled = tree_labeled



def displacy_dep_input(sent):
    deps = []
    for dep in sent.split('\n'):
        deps.append(dep.split())

    deps = [x for x in deps if x]

    ex = []
    word_list = []
    arc_list = []

    for index, dep in enumerate(deps):
        word_list.append({"text": dep[0], "tag": ""})
        line = index+1
        head = int(dep[2])
        label = dep[3]
        if head>line:
            start = index
            end = head-1
            direction = "left"
        else:
            start = head-1
            end = index  
            direction = "right"
        if(label.lower() != "root"):
            arc_list.append({"start": start, "end": end, "label": label, "dir": direction})

    ex.append({
        "words": word_list,
        "arcs": arc_list
    })    

    return ex



from nltk.featstruct import Feature, UnificationFailure, FeatStructReader
import itertools
from collections import defaultdict


def check_sanity_constraints(th):
    for type1, type2 in itertools.product(th, th):
        if type1 in th[type2] and type2 in th[type1]:
            if type1 != type2:
                raise ValueError(
                    "The type hierarchy is not antisymmetric! " +
                    "{} subsumes {} and vice versa!".format(
                        type1, type2
                    )
                )


def refl_trans_closure(type_hierarchy):
    # make everything a set
    # and compute reflexive closure
    closure = defaultdict(set)
    for t in type_hierarchy:
        closure[t] = set(type_hierarchy[t])
        closure[t].add(t)

    # compute transitive closure
    still_changes = True
    while still_changes:
        still_changes = False
        for x in closure:
            new_for_x = set()
            for y in closure[x]:
                for z in closure[y]:
                    new_for_x.add(z)
            len_before = len(closure[x])
            closure[x].update(new_for_x)
            still_changes |= len(closure[x]) > len_before

    return closure


class HierarchicalFeature(Feature):
    def __init__(self, name, type_hierarchy, **kwargs):
        super(HierarchicalFeature, self).__init__(name, **kwargs)

        self.hierarchy = refl_trans_closure(type_hierarchy)
        check_sanity_constraints(self.hierarchy)

    def unify_base_values(self, fval1, fval2, bindings):
        candidates = self.hierarchy[fval1].intersection(self.hierarchy[fval2])
        score = {t: 0 for t in candidates}
        for type1, type2 in itertools.product(candidates, candidates):
            if type1 in self.hierarchy[type2]:
                score[type1] += 1

        return min(candidates, key=score.__getitem__, default=UnificationFailure)

--- 



[zurück zur Übersicht](#Übersicht)

---

## 1. Ambiguität in Konstituentengrammatik


### Gegeben sei folgender Satz des Englischen (*lower case*, ohne Satzzeichen): 

***will will will the will***

### Schreiben Sie zu diesem *Beispielsatz für syntaktische Ambiguität*  eine minimale CFG, die genau die intendierte Struktur des Satzes erkennt.

### Folgende *intendierte Struktur* soll dabei gemäß Ihrer Gruppennummer modelliert werden:


#### `Gruppe 1` > **Fragesatz:**  *Wird Will das Testament vererben?*
#### `Gruppe 2` und `Gruppe 3` > **Aussagesatz:**  *Will wird das Testament vererben.*



### Testen Sie anschließend Ihre Grammatik (nur die erwünschte Analyse wird ausgegeben).


In [ ]:
#TO_DO:
sentence = "will will will the will"

grammar = nltk.CFG.fromstring("""


PROPN -> 'will'
N -> 'will'
V -> 'will'
AUX -> 'will'
DET -> 'the'
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (1):*
<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sentence = "will will will the will"

grammar = nltk.CFG.fromstring("""


PROPN -> 'will'
N -> 'will'
V -> 'will'
AUX -> 'will'
DET -> 'the'
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)
</code>
</div>

---

## 2. Ambiguität in Dependenzgrammatik

### Gegeben sei wieder folgender Satz des Englischen

***will will will the will***

### sowie eine ungelabelte Dependenzgrammatik, die aufgrund der lexikalischen Ambiguität von *will* eine große Menge an Ableitungen produziert.

### Wählen sie über den Listenindex (an Stelle des TODO)  aus den ersten 5 Ableitungsbäumen der Grammatik den Baum aus, der folgende Struktur modelliert (Zuordnung gemäß Gruppennummer):


#### `Gruppe 1` > **Aussagesatz:**  *Will wird das Testament vererben.*
#### `Gruppe 2` und `Gruppe 3` > **Fragesatz:**  *Wird Will das Testament vererben?*


### *Dabei soll der Baum den TIGER-Dependenzregeln entsprechen!*


### Führen Sie dazu zunächst folgende Codezelle aus:


In [4]:
#RUN (Führen Sie zunächst diese Code-Zelle aus):
sentence = "will will will the will"

grammar = nltk.DependencyGrammar.fromstring("""
    'will' -> 'will'
    'will' -> 'the'
""")

parser = nltk.ProjectiveDependencyParser(grammar)
trees = list(parser.parse(sentence.split()))

for (i, tree) in enumerate(trees):
    print(i, tree, "\n")
    tree.pretty_print(unicodelines=True)
    if i > 4: break

0 (will (will (will will) the)) 

    will     
     │        
    will     
 ┌───┴────┐   
 │       will
 │        │   
the      will

1 (will will (will (will the))) 

     will     
 ┌────┴────┐   
 │        will
 │         │   
 │        will
 │         │   
will      the 

2 (will (will will) the will) 

    will     
 ┌───┼────┐   
 │   │   will
 │   │    │   
the will will

3 (will (will (will will)) the) 

    will     
 ┌───┴────┐   
 │       will
 │        │   
 │       will
 │        │   
the      will

4 (will (will (will will) the)) 

    will     
     │        
    will     
 ┌───┴────┐   
 │       will
 │        │   
the      will

5 (will will will (will the)) 

     will     
 ┌────┼────┐   
 │    │   will
 │    │    │   
will will the 



In [ ]:
#TO_DO:
tree = trees[TODO]
tree.pretty_print(unicodelines=True)

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (2):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
tree = trees[TODO]
tree.pretty_print(unicodelines=True)
</code>
</div>

--- 

## 3. Konstituenten- und Adjunkttests

## 3.1 Eliminierungstest

#### Gegeben sei folgender Satz sowie die Resulte der systematischen Elimierung eines Wortes:


In [5]:
#RUN (Führen Sie zunächst diese Code-Zelle aus):
sentence = ["er", "schreibt", "es", "ihr", "heute"]

sentencelist = []
for i in range(len((sentence))):
    sentencelist.append(sentence.copy())
    sentencelist[i].pop(i)
    print(i, sentencelist[i])

0 ['schreibt', 'es', 'ihr', 'heute']
1 ['er', 'es', 'ihr', 'heute']
2 ['er', 'schreibt', 'ihr', 'heute']
3 ['er', 'schreibt', 'es', 'heute']
4 ['er', 'schreibt', 'es', 'ihr']


### Führen Sie obenstehende Codezelle aus.

### Geben Sie über den Listenindex (an Stelle des TODO) eine Version des Satzes mit Eliminierung an, welche folgendes Satzglied als fakultativen bzw. optionalen Dependenten feststellt (Zuordnung gemäß Gruppennummer):


#### `Gruppe 1` > Objekt
#### `Gruppe 2` > indirektes Objekt
#### `Gruppe 3` > Adverbial


In [ ]:
#TO_DO:
sentencelist[TODO]

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (3.1):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sentencelist[TODO]
</code>
</div>

---

## 3.2 Geschehenstest


#### Gegeben sei wieder obiger Satz:

In [ ]:
sentence = ["er", "schreibt", "es", "ihr", "heute"]

### Geben Sie über den Listenindex (an Stelle des TODO) ein eliminierbares Satzglied an, das über den Geschehenstest als  Komplement festgestellt wird:


In [ ]:
#TO_DO:
sentence = ["er", "schreibt", "es", "ihr", "heute"]
sentence[0] + " " + sentence[1] + ", und das geschieht " + sentence[TODO]

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (3.2):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sentence = ["er", "schreibt", "es", "ihr", "heute"]
sentence[0] + " " + sentence[1] + ", und das geschieht " + sentence[TODO]
</code>
</div>

---

## 4. CFG-Analysen

#### Gegeben sei folgender Satz, zusätzlich eine rudimentäre Grammatik:


In [6]:
sentence = "er schreibt"

grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    VP  -> VERBAL
    VERBAL -> V
    NP  -> PRON

    PRON -> "er" | "es" | "ihr"
    V    -> "schreibt"    
    ADV  -> "heute"
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

      S          
 ┌────┴─────┐     
 │          VP   
 │          │     
 NP       VERBAL 
 │          │     
PRON        V    
 │          │     
 er      schreibt



### 4a) Ergänzen Sie den Satz der Angabe mit den in der Grammatik gegebenen Terminalen, so dass er ein: 


#### `Gruppe 1` > indirektes Objekt
#### `Gruppe 2` > Adverbial
#### `Gruppe 3` > Objekt

### enthält (Zuordnung gemäß Gruppennummer).


In [ ]:
#TO_DO:
sentence = "er schreibt TODO"

### 4b) Erweitern Sie die Grammatik um entsprechende syntaktische Regeln für den erweiterten Satz.  Halten Sie sich dabei in den ergänzten verbalen Regeln an das X-Bar-Schema!

#### Beachten Sie, dass Sie zuvor die Codezelle von 4a) ausgeführt haben müssen.

In [7]:
#TO_DO:
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    VP  -> VERBAL
    VERBAL -> V
    NP  -> PRON

    PRON -> "er" | "es" | "ihr"
    V    -> "schreibt"    
    ADV  -> "heute"

###########ERGAENZTE REGELN:    
    
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

      S          
 ┌────┴─────┐     
 │          VP   
 │          │     
 NP       VERBAL 
 │          │     
PRON        V    
 │          │     
 er      schreibt



---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (4a):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sentence = "er schreibt TODO"
</code>
</div>

*Ursprüngliche Angabe (4b):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    VP  -> VERBAL
    VERBAL -> V
    NP  -> PRON

    PRON -> "er" | "es" | "ihr"
    V    -> "schreibt"    
    ADV  -> "heute"

###########ERGAENZTE REGELN:    
    
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    
</code>
</div>

---

## 5. Dependenzanalyse

### Analysieren Sie die Dependenzbeziehungen untenstehender Sätze im UD-Schema (Zuordnung gemäß Gruppennummer). Verwenden Sie das aus der Vorlesung bekannte Format: 

- pro Zeile: `Position, Wort, Position des Kopfes, Dependenzrelation`
- Wurzelknoten: `Position des Kopfes` = 0, `Dependenzrelation` = ROOT



#### `Gruppe 1` > ***Josef Konrad ist Prokurist einer Bank***
#### `Gruppe 2` > ***Der Wächter Franz verkündet ihm seine Verhaftung***
#### `Gruppe 3` > ***Frau Grubach ist Josefs Vermieterin***

In [8]:
#TO_DO:
sent_nr = """
1 x 2 dep
2 y 0 ROOT
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

tree_labeled = dg.tree_labeled()
tree_labeled.pretty_print(unicodelines=True)  

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

y(ROOT)
   │    
 x(dep)



---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (5):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sent_nr = """
1 x 2 dep
2 y 0 ROOT
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

tree_labeled = dg.tree_labeled()
tree_labeled.pretty_print(unicodelines=True)  

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})
</code>
</div>

---

## 6. Feature-based-Grammar

#### Gegeben sei folgende Grammatik mit VP-Regeln für transitive und intransitive Verben sowie einer Regel für rekursives Attachment von NP-Adjunkten an VPs:

In [9]:
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    VP  -> V NP
    VP  -> V
    VP  -> VP NP
    NP  -> DET N
    NP  -> DET ADJ N 

    DET -> "der" | "den"
    ADJ -> "ganzen"
    N   -> "Hund" | "Briefträger" | "Tag"
    V   -> "jagt" | "rennt" 
""")

parser = nltk.ChartParser(grammar,trace=0)

## 6.1 Komplement vs. Adjunkt

#### Die Grammatik erkennt u.a. folgende, grammatisch korrekte Sätze, aber fürt jeweils zu Übergenerierung:

In [10]:
sentence = "der Hund rennt den ganzen Tag"
for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

               S                     
     ┌─────────┴────────┐             
     │                  VP           
     │         ┌────────┴────┐        
     NP        VP            NP      
 ┌───┴───┐     │    ┌────────┼─────┐  
DET      N     V   DET      ADJ    N 
 │       │     │    │        │     │  
der     Hund rennt den     ganzen Tag

               S                     
     ┌─────────┴────────┐             
     │                  VP           
     │         ┌────────┴────┐        
     NP        │             NP      
 ┌───┴───┐     │    ┌────────┼─────┐  
DET      N     V   DET      ADJ    N 
 │       │     │    │        │     │  
der     Hund rennt den     ganzen Tag



In [11]:
sentence = "der Hund jagt den Briefträger"
for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

              S                      
     ┌────────┴────┐                  
     │             VP                
     │        ┌────┴───┐              
     NP       VP       NP            
 ┌───┴───┐    │    ┌───┴───────┐      
DET      N    V   DET          N     
 │       │    │    │           │      
der     Hund jagt den     Briefträger

              S                      
     ┌────────┴────┐                  
     │             VP                
     │        ┌────┴───┐              
     NP       │        NP            
 ┌───┴───┐    │    ┌───┴───────┐      
DET      N    V   DET          N     
 │       │    │    │           │      
der     Hund jagt den     Briefträger



### Erweitern Sie die angegebene Grammatik um die notwendigen Merkmale, um für die beiden Sätze jeweils die Überproduktion zu verhindern.

In [ ]:
#TO_DO:
gramstring = r"""
% start S

    S   -> NP VP
    VP  -> V NP
    VP  -> V
    VP  -> VP NP
    NP  -> DET N
    NP  -> DET ADJ N 

    DET -> "der" | "den"
    ADJ -> "ganzen"
    N   -> "Hund" | "Briefträger" | "Tag"
    V   -> "jagt" | "rennt" 
"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

sentence = "der Hund rennt den ganzen Tag"
for tree in parser.parse(sentence.split()):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
    
sentence = "der Hund jagt den Briefträger"
for tree in parser.parse(sentence.split()):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (6.1):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
gramstring = r"""
% start S

    S   -> NP VP
    VP  -> V NP
    VP  -> V
    VP  -> VP NP
    NP  -> DET N
    NP  -> DET ADJ N 

    DET -> "der" | "den"
    ADJ -> "ganzen"
    N   -> "Hund" | "Briefträger" | "Tag"
    V   -> "jagt" | "rennt" 
"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

sentence = "der Hund rennt den ganzen Tag"
for tree in parser.parse(sentence.split()):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
    
sentence = "der Hund jagt den Briefträger"
for tree in parser.parse(sentence.split()):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
</code>
</div>

## 6.2 Kasus-Feature

#### Folgende Grammatik enthält Regeln für transitive Verben, inkl. Regeln für eine invertierte Worstellung (d.h. mit Subjekt-NP satzfinal).

### Erweitern Sie diese Grammatik um die notwendigen Merkmale, um auch im folgenden Beispielsatz die Überproduktion auszuschließen:



In [13]:
grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    VP  -> VP NP

    S   -> VP NP
    VP  -> NP VP
    
    VP  -> V

    NP  -> DET N
    NP  -> DET ADJ N 

    DET -> "der" | "den"
    ADJ -> "ganzen"
    N   -> "Hund" | "Briefträger" | "Tag"
    V   -> "rennt" 
""")

parser = nltk.ChartParser(grammar,trace=0)

sentence = "den ganzen Tag rennt der Hund"
for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)

                S                    
            ┌───┴─────────────┐       
            VP                │      
      ┌─────┴────────┐        │       
      NP             VP       NP     
 ┌────┼─────────┐    │    ┌───┴───┐   
DET  ADJ        N    V   DET      N  
 │    │         │    │    │       │   
den ganzen     Tag rennt der     Hund

            S                    
      ┌─────┴─────────┐           
      │               VP         
      │          ┌────┴───┐       
      NP         VP       NP     
 ┌────┼─────┐    │    ┌───┴───┐   
DET  ADJ    N    V   DET      N  
 │    │     │    │    │       │   
den ganzen Tag rennt der     Hund



In [14]:
#TO_DO:
gramstring = r"""
% start S

    S   -> NP VP
    VP  -> VP NP

    S   -> VP NP
    VP  -> NP VP
    
    VP  -> V
 
    NP  -> DET N
    NP  -> DET ADJ N 

    DET -> "der" | "den"
    ADJ -> "ganzen"
    N   -> "Hund" | "Briefträger" | "Tag"
    V   -> "rennt" 
"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

sentence = "den ganzen Tag rennt der Hund"
for tree in parser.parse(sentence.split()):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)

                  S[]                      
              ┌────┴───────────────┐        
             VP[]                  │       
        ┌─────┴─────────┐          │        
       NP[]            VP[]       NP[]     
  ┌─────┼──────────┐    │     ┌────┴────┐   
DET[] ADJ[]       N[]  V[]  DET[]      N[] 
  │     │          │    │     │         │   
 den  ganzen      Tag rennt  der       Hund

             S[]                      
        ┌─────┴──────────┐             
        │               VP[]          
        │          ┌─────┴────┐        
       NP[]       VP[]       NP[]     
  ┌─────┼─────┐    │     ┌────┴────┐   
DET[] ADJ[]  N[]  V[]  DET[]      N[] 
  │     │     │    │     │         │   
 den  ganzen Tag rennt  der       Hund



---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (6.2):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
gramstring = r"""
% start S

    S   -> NP VP
    VP  -> VP NP

    S   -> VP NP
    VP  -> NP VP
    
    VP  -> V
 
    NP  -> DET N
    NP  -> DET ADJ N 

    DET -> "der" | "den"
    ADJ -> "ganzen"
    N   -> "Hund" | "Briefträger" | "Tag"
    V   -> "rennt" 
"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

sentence = "den ganzen Tag rennt der Hund"
for tree in parser.parse(sentence.split()):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
</code>
</div>

---

## 7. Slash-Kategorien



#### Gegeben sei folgender Satz und eine FCFG, die das Herausbewegen des Fragepronomens des Subjektsatzes als COMP der SBAR-Regel modelliert:


In [15]:
sentence = "wer es glaubt wird selig"

gramstring = r"""
% start S
    S -> SBAR VP
    VP -> COP ADJ
    
    SBAR -> COMP S/NP
    S/?x -> NP/?x VP
    NP/NP -> 
    
    VP -> NP V

    NP -> PRON
    
    COMP -> "wer"
    V   -> "glaubt"
    PRON   -> "es" 
    COP  -> "wird"
    ADJ -> "selig"

"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)

                          S[]                                
                   ┌───────┴────────────────────────┐         
                 SBAR[]                             │        
  ┌────────────────┴───────┐                        │         
  │                     S[]/NP[]                    │        
  │        ┌───────────────┴──────┐                 │         
  │        │                     VP[]               │        
  │        │               ┌──────┴─────┐           │         
  │        │              NP[]          │          VP[]      
  │        │               │            │      ┌────┴─────┐   
COMP[] NP[]/NP[]         PRON[]        V[]   COP[]      ADJ[]
  │        │               │            │      │          │   
 wer      ...              es         glaubt  wird      selig



### Kopieren Sie die Regel(n) aus obenstehender Grammatik herunter, die das folgende implementiert (Zuordnung gemäß Gruppennummer):



#### `Gruppe 1` > Herunterreichen der Gap-Information
#### `Gruppe 2` > Gap-Introduction
#### `Gruppe 3` > Gap-Realisierung




In [ ]:
#TO_DO:
gramstring = r"""

"""

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (7):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
gramstring = r"""

"""
</code>
</div>

---

## 8. Syntaxregeln komplexer Sätze

## 8.1 CFG-Regeln für komplexe Sätze

#### Gegeben sei folgender Satz sowie eine CFG-Grammatik, die u.a. diesen Satz erkennt:



In [16]:
sentence = "er glaubt zu träumen"

grammar = nltk.CFG.fromstring("""
    S    -> NP VP
    NP   -> PRON
    
    VP   -> V VP    
    VP   -> TO V 

    PRON -> "er"
    V    -> "glaubt" | "glauben" | "träumt" |"träumen" | "fliegt" | "fliegen"
    TO   -> "zu"
    COMP -> "dass"
    COMP ->
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    
    

             S                 
 ┌───────────┴───┐              
 │               VP            
 │     ┌─────────┴───┐          
 NP    │             VP        
 │     │         ┌───┴─────┐    
PRON   V         TO        V   
 │     │         │         │    
 er  glaubt      zu     träumen



### 8.1 a) Geben Sie einen neuen Satz an, der einen *finiten Objektsatz* enthält. 

### Greifen Sie dabei ausschließlich auf die in der Grammatik vorhandenen Lexeme zurück!


#### (Satzzeichen müssen nicht berücksichtigt werden!)


In [ ]:
#TO_DO:
sentence = ""

---

[zurück zur Übersicht](#Übersicht)


### 8.1 b) Erweitern Sie die Grammatik um entsprechende syntaktische Regeln für Ihren  Satz aus 8.1a.


- Verwenden Sie nur die bestehenden sowie ggf. `SBAR` und `COMP` als neue Nonterminale
- X-Bar-Schema ist nicht notwendig (orientieren Sie sich an den Penn-Treebank-Regeln für komplexe Sätze)
- Beachten Sie die invertierte Wortstellung im Nebensatz (Verbendstellung).

#### Beachten Sie, dass Sie zuvor die Codezelle von 8.1a) ausgeführt haben müssen.


In [ ]:
#TO_DO:
grammar = nltk.CFG.fromstring("""
    S    -> NP VP
    NP   -> PRON
    
    VP   -> V VP    
    VP   -> TO V 

    PRON -> "er"
    V    -> "glaubt" | "glauben" | "träumt" |"träumen" | "fliegt" | "fliegen"
    TO   -> "zu"
    COMP -> "dass"
    COMP ->

###########ERGAENZTE REGELN:    

""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (8.1a):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sentence = ""
</code>
</div>

*Ursprüngliche Angabe (8.1b):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
grammar = nltk.CFG.fromstring("""
    S    -> NP VP
    NP   -> PRON
    
    VP   -> V VP    
    VP   -> TO V 

    PRON -> "er"
    V    -> "glaubt" | "glauben" | "träumt" |"träumen" | "fliegt" | "fliegen"
    TO   -> "zu"
    COMP -> "dass"
    COMP ->

###########ERGAENZTE REGELN:    

""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    
</code>
</div>

---

## 8.2. Analysieren Sie die Dependenzbeziehungen Ihres Beispielsatzes aus 8.1 im UD-Schema


In [18]:
#TO_DO:
sent_nr = """
1 x 2 dep
2 y 0 ROOT
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

tree_labeled = dg.tree_labeled()
tree_labeled.pretty_print(unicodelines=True)  

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

y(ROOT)
   │    
 x(dep)



---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (8.2):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sent_nr = """
1 x 2 dep
2 y 0 ROOT
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

tree_labeled = dg.tree_labeled()
tree_labeled.pretty_print(unicodelines=True)  

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})
</code>
</div>

---

## 9. Parsing-Algorithmen und Rekursionstypen

### 9.1 Kommentieren Sie in folgender Grammatik genau eine Regel aus, so dass der verwendete Shift-Reduce-Parser auch ohne Backtracking eine Ableitung findet:


In [29]:
#TO_DO:
sentence = "sie beobachtet den Hund aus dem Versteck"

grammar = nltk.CFG.fromstring("""
S -> NP VP

VP -> V NP PP
VP -> V NP

NP -> DET N
NP -> PRON

PP -> P NP

PRON -> "sie"
V -> "beobachtet"
DET -> "den"
N -> "Hund"
P -> "aus"
DET -> "dem"
N -> "Versteck"
""")

parser = nltk.ShiftReduceParser(grammar,trace=2)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

Parsing 'sie beobachtet den Hund aus dem Versteck'
    [ * sie beobachtet den Hund aus dem Versteck]
  S [ 'sie' * beobachtet den Hund aus dem Versteck]
  R [ PRON * beobachtet den Hund aus dem Versteck]
  R [ NP * beobachtet den Hund aus dem Versteck]
  S [ NP 'beobachtet' * den Hund aus dem Versteck]
  R [ NP V * den Hund aus dem Versteck]
  S [ NP V 'den' * Hund aus dem Versteck]
  R [ NP V DET * Hund aus dem Versteck]
  S [ NP V DET 'Hund' * aus dem Versteck]
  R [ NP V DET N * aus dem Versteck]
  R [ NP V NP * aus dem Versteck]
  R [ NP VP * aus dem Versteck]
  R [ S * aus dem Versteck]
  S [ S 'aus' * dem Versteck]
  R [ S P * dem Versteck]
  S [ S P 'dem' * Versteck]
  R [ S P DET * Versteck]
  S [ S P DET 'Versteck' * ]
  R [ S P DET N * ]
  R [ S P NP * ]
  R [ S PP * ]


---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (9.1):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sentence = "sie beobachtet den Hund aus dem Versteck"

grammar = nltk.CFG.fromstring("""
S -> NP VP

VP -> V NP PP
VP -> V NP

NP -> DET N
NP -> PRON

PP -> P NP

PRON -> "sie"
V -> "beobachtet"
DET -> "den"
N -> "Hund"
P -> "aus"
DET -> "dem"
N -> "Versteck"
""")

parser = nltk.ShiftReduceParser(grammar,trace=2)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)
</code>
</div>

--- 

### 9.2 Geben Sie zu der folgenden CFG-Regel eine weitere Regel an, die in Kombination mit dieser zu Problemen bei der Verarbeitung mit einem Recursive-Descent-Parser führt.

In [20]:
#TO_DO:
grammar = nltk.CFG.fromstring("""
    NP -> X PP

""")

print(grammar)

Grammar with 1 productions (start state = NP)
    NP -> X PP


---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (9.2):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
grammar = nltk.CFG.fromstring("""
    NP -> X PP

""")

print(grammar)
</code>
</div>

---

## 10. Unifikationsparsing und getypte Merkmalstrukturen





## 10.1 Unifikation und Merkmalsstrukturen

#### Gegeben seien folgende (unifizierende) Merkmalsstrukturen:

In [92]:
f1 = FeatStruct("[+X]")
f2 = FeatStruct("[]")
print(f1.unify(f2))

[ X = True ]



### Geben Sie eine *nicht-leere* Merkmalsstruktur `f2` an, die den boolschen Feature-Wert `False` enthält und die mit `f1`unifiziert.

In [ ]:
#TO_DO:
f1 = FeatStruct("[+X]")
f2 = FeatStruct("[]")
print(f1.unify(f2))

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (10.1):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
f1 = FeatStruct("[+X]")
f2 = FeatStruct("[]")
print(f1.unify(f2))
</code>
</div>

---


## 10.2 Unifikation mit Typen

#### Gegeben sei folgende Agreement-Typhierarchie, die (mit abgekürzten Typnamen) durch das `*AGR*`-Feature implementiert wird:

In [ ]:
#RUN (Führen Sie zunächst diese Code-Zelle aus):
type_hierarchy = {
    "1": ["1sg","1pl"],
    "2": ["2sg", "2pl"],
    "3": ["3sg", "3pl"],
    "sg": ["1sg", "2sg", "3sg"],
    "pl": ["1pl", "2pl", "3pl"],
    "1sg": [],
    "1pl": [],
    "2sg": [],
    "2pl": [],
    "3sg": [],
    "3pl": []
}
AGR = HierarchicalFeature("AGR", type_hierarchy)
reader = FeatStructReader(features=(AGR,))

### Führen Sie obenstehende Codezelle aus, um die Typhierarchie zu laden.

### Geben Sie zwei *nicht-leere* Merkmalstrukturen  `f1` und `f2` an, die von `f3`verschieden sind, sodass gilt:

`f1` subsumiert `f2`, `f2` subsumiert `f3`.

In [ ]:
#TO_DO:
f1 = reader.fromstring("[]")
f2 = reader.fromstring("[]")
f3 = reader.fromstring("[*AGR*='2sg']")
print(f1.subsumes(f2), f2.subsumes(f3)) 

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (10.2):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
f1 = reader.fromstring("[]")
f2 = reader.fromstring("[]")
f3 = reader.fromstring("[*AGR*='2sg']")
print(f1.subsumes(f2), f2.subsumes(f3)) 
</code>
</div>

---

## 11. Statistisches Parsing


## 11.1 PCFG: Gewichte und Ableitungswahrscheinlichkeit



#### Gegeben sei folgende Mini-Treebank mit PP-Attachment-Sätzen:


In [21]:
treestrings = [
"(S (NP Ich) (VP (V gehe)) (PP auf dem Weg))",
"(S (NP Ich) (VP (V steige) (PP auf den Berg)))",
"(S (NP Ich) (VP (V klettere) (PP auf den Berg)))",
]

trees = []
for treestring in treestrings:
    trees.append(Tree.fromstring(treestring))
    
for tree in trees:
    tree.pretty_print(unicodelines=True)

          S         
 ┌───┬────┴───┐      
 │   VP       │     
 │   │        │      
 NP  V        PP    
 │   │    ┌───┼───┐  
Ich gehe auf dem Weg

            S              
 ┌──────────┴───┐           
 │              VP         
 │    ┌─────────┴───┐       
 NP   V             PP     
 │    │     ┌───────┼───┐   
Ich steige auf     den Berg

              S              
 ┌────────────┴───┐           
 │                VP         
 │     ┌──────────┴───┐       
 NP    V              PP     
 │     │      ┌───────┼───┐   
Ich klettere auf     den Berg



### Passen Sie diese Mini-Treebank durch Umklammern der syntaktischen Struktur einer minimalen Anzahl an Sätzen so an, dass die daraus induzierte Grammatik das S-Attachment (Satz-Adjunkt statt Subkategorisierung nach PP-Komplement) bevorzugt.

In [22]:
#TO_DO:
treestrings = [
"(S (NP Ich) (VP (V gehe)) (PP auf dem Weg))",
"(S (NP Ich) (VP (V steige) (PP auf den Berg)))",
"(S (NP Ich) (VP (V klettere) (PP auf den Berg)))",
]

trees = []
for treestring in treestrings:
    trees.append(Tree.fromstring(treestring))
    
    
#grammar induction:    
productions = []
S = nltk.Nonterminal('S')

for tree in trees:
    productions += tree.productions()

grammar = nltk.induce_pcfg(S, productions)
for production in grammar.productions():
    print(production)    
    
#parse trees with grammar:    
parser = nltk.ViterbiParser(grammar)
     
for tree in trees:
    for parse in parser.parse(tree.leaves()): 
        print(parse)
        parse.pretty_print(unicodelines=True) 

S -> NP VP PP [0.333333]
NP -> 'Ich' [1.0]
VP -> V [0.333333]
V -> 'gehe' [0.333333]
PP -> 'auf' 'dem' 'Weg' [0.333333]
S -> NP VP [0.666667]
VP -> V PP [0.666667]
V -> 'steige' [0.333333]
PP -> 'auf' 'den' 'Berg' [0.666667]
V -> 'klettere' [0.333333]
(S (NP Ich) (VP (V gehe) (PP auf dem Weg))) (p=0.0493827)
          S             
 ┌────────┴───┐          
 │            VP        
 │   ┌────────┴───┐      
 NP  V            PP    
 │   │    ┌───────┼───┐  
Ich gehe auf     dem Weg

(S (NP Ich) (VP (V steige) (PP auf den Berg))) (p=0.0987654)
            S              
 ┌──────────┴───┐           
 │              VP         
 │    ┌─────────┴───┐       
 NP   V             PP     
 │    │     ┌───────┼───┐   
Ich steige auf     den Berg

(S (NP Ich) (VP (V klettere) (PP auf den Berg))) (p=0.0987654)
              S              
 ┌────────────┴───┐           
 │                VP         
 │     ┌──────────┴───┐       
 NP    V              PP     
 │     │      ┌───────┼───┐   
Ich 

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (11.1):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
treestrings = [
"(S (NP Ich) (VP (V gehe)) (PP auf dem Weg))",
"(S (NP Ich) (VP (V steige) (PP auf den Berg)))",
"(S (NP Ich) (VP (V klettere) (PP auf den Berg)))",
]

trees = []
for treestring in treestrings:
    trees.append(Tree.fromstring(treestring))
    
    
#grammar induction:    
productions = []
S = nltk.Nonterminal('S')

for tree in trees:
    productions += tree.productions()

grammar = nltk.induce_pcfg(S, productions)
for production in grammar.productions():
    print(production)    
    
#parse trees with grammar:    
parser = nltk.ViterbiParser(grammar)
     
for tree in trees:
    for parse in parser.parse(tree.leaves()): 
        print(parse)
        parse.pretty_print(unicodelines=True) 
</code>
</div>

---

## 11.2 Übergangsbasierter Shift-Reduce-Dependency-Parser

#### Gegeben sei folgender Dependenzgraph:

In [128]:
sent_nr = """
1 ein 3 det 
2 seltenes 3 amod
3 Tier 5 nsubj
4 wurde 5 aux
5 gesehen 0 ROOT
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

### Geben Sie den Typ der REDUCE-Übergänge (`LEFTARC`, `RIGHTARC`) sowie die Reihenfolge deren Durchführung bei Verarbeitung dieser Struktur mit einem Shift-Reduce-Dependency-Parser an.


In [23]:
#TO_DO:
sent_nr = """
1 ein 3 LEFTARC-?
2 seltenes 3 ?
3 Tier 5 ?
4 wurde 5 ?
5 gesehen 0 ROOT
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (11.2):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sent_nr = """
1 ein 3 LEFTARC-?
2 seltenes 3 ?
3 Tier 5 ?
4 wurde 5 ?
5 gesehen 0 ROOT
"""

sent = transform_nr_conll(sent_nr)
dg = DependencyGraph(sent)

ex = displacy_dep_input(sent)
html = displacy.render(ex, style="dep", manual=True, options={'distance':100})
</code>
</div>

---

## 12. Datengestützte Syntaxanalyse




## 12.1 Lexikalisierte CFG (mit Merkmalen)


### Führen Sie in folgender FCFG über ein HEAD-Merkmal eine vollständige Kopfannotation durch.

#### Berücksichtigen Sie dabei das *Primacy-of-Content-Words* Paradigma von UD.


In [24]:
#TO_DO:
sentence = "er träumt auf den Berg geflogen zu sein"

gramstring = r"""
% start S
    S[]   -> NP[] VP[]
    NP[]  -> DET[] N[]
    NP[]  -> N[]
    PP[]  -> P[] NP[]
    VP[]  -> V[] VP[]
    VP[]  -> PP[] V[] TO[] AUX[]

    DET[HEAD=""] -> "den"
    N[]   -> "er"    
    N[]   -> "Berg"
    P[]   -> "auf"
    V[]   -> "träumt"   
    V[]   -> "geflogen"    
    AUX[] -> "sein"
    TO[]  -> "zu"
"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    print(tree)    
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
    #display(tree)

(S[]
  (NP[] (N[] er))
  (VP[]
    (V[] träumt)
    (VP[]
      (PP[] (P[] auf) (NP[] (DET[HEAD=''] den) (N[] Berg)))
      (V[] geflogen)
      (TO[] zu)
      (AUX[] sein))))
      S[]                                                  
 ┌─────┴─────┐                                              
 │          VP[]                                           
 │     ┌─────┴──────────────────────┐                       
 │     │                           VP[]                    
 │     │              ┌─────────────┴──────┬──────┬─────┐   
 │     │             PP[]                  │      │     │  
 │     │     ┌────────┴────────┐           │      │     │   
NP[]   │     │                NP[]         │      │     │  
 │     │     │        ┌────────┴────┐      │      │     │   
N[]   V[]   P[]  DET[HEAD='']      N[]    V[]    TO[] AUX[]
 │     │     │        │             │      │      │     │   
 er  träumt auf      den           Berg geflogen  zu   sein



---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (12.1):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sentence = "er träumt auf den Berg geflogen zu sein"

gramstring = r"""
% start S
    S[]   -> NP[] VP[]
    NP[]  -> DET[] N[]
    NP[]  -> N[]
    PP[]  -> P[] NP[]
    VP[]  -> V[] VP[]
    VP[]  -> PP[] V[] TO[] AUX[]

    DET[HEAD=""] -> "den"
    N[]   -> "er"    
    N[]   -> "Berg"
    P[]   -> "auf"
    V[]   -> "träumt"   
    V[]   -> "geflogen"    
    AUX[] -> "sein"
    TO[]  -> "zu"
"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    print(tree)    
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
    #display(tree)
</code>
</div>


---


## 12.2 Parent Annotation (mit Symbolerweiterung)

### Gegeben sei wieder obiger Satz mit entsprechender CFG.

### Führen Sie über Symbolerweiterung (mit `^` als Trennerzeichen) in der CFG eine vollständige *Parent Annotation* durch, wie Sie durch die Regelanwendungen im Syntaxbaum der Angabe impliziert ist. 

### Ergänzen Sie ggf. auch Regeln, die für die vollständige Parent-Annotation notwendig sind.


In [25]:
#TO_DO:
sentence = "er träumt auf den Berg geflogen zu sein"

grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    NP  -> DET N
    NP  -> N
    PP  -> P NP
    VP  -> V VP
    VP  -> PP V TO AUX

    DET -> "den"
    N   -> "er"    
    N   -> "Berg"
    P   -> "auf"
    V   -> "träumt"   
    V   -> "geflogen"    
    AUX -> "sein"
    TO  -> "zu"

###########ERGAENZTE REGELN:    
    
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    

      S                                      
 ┌────┴─────┐                                 
 │          VP                               
 │    ┌─────┴───────────┐                     
 │    │                 VP                   
 │    │         ┌───────┴──────┬──────┬───┐   
 │    │         PP             │      │   │  
 │    │     ┌───┴───┐          │      │   │   
 NP   │     │       NP         │      │   │  
 │    │     │   ┌───┴───┐      │      │   │   
 N    V     P  DET      N      V      TO AUX 
 │    │     │   │       │      │      │   │   
 er träumt auf den     Berg geflogen  zu sein



---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (12.2):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
sentence = "er träumt auf den Berg geflogen zu sein"

grammar = nltk.CFG.fromstring("""
    S   -> NP VP
    NP  -> DET N
    NP  -> N
    PP  -> P NP
    VP  -> V VP
    VP  -> PP V TO AUX

    DET -> "den"
    N   -> "er"    
    N   -> "Berg"
    P   -> "auf"
    V   -> "träumt"   
    V   -> "geflogen"    
    AUX -> "sein"
    TO  -> "zu"

###########ERGAENZTE REGELN:    
    
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)    
</code>
</div>

---

## 13. Chunk-Analysen



### Geben Sie die *IOB-Tag-Sequenz* einer *flachen* VP-NP-Chunk-Analyse für den folgenden Satz an  (Zuordnung gemäß Gruppennummer):

#### `Gruppe 1` > ***Josef Konrad ist Prokurist einer Bank***
#### `Gruppe 2` > ***Der Wächter Franz verkündet ihm seine Verhaftung***
#### `Gruppe 3` > ***Frau Grubach ist Josefs Vermieterin***

### Schreiben Sie dazu in der unten gegebenen Liste für jedes Wort Ihres Satzes ein Tripel der Form `(WORT, POS-TAG, IOB-TAG)` (vgl. Angabe).


#### Sie können ein beliebiges POS-Tagset verwenden.

In [27]:
#TO_DO:
iob_list = [
("ein", "DET", "B-NP"),
("Beispiel", "N", "I-NP"),
]

from nltk import conlltags2tree
tree = conlltags2tree(iob_list)
tree.pretty_print(unicodelines=True)

       S            
       │             
       NP           
  ┌────┴──────┐      
ein/N     Beispiel/N



---

[zurück zur Übersicht](#Übersicht)


*Ursprüngliche Angabe (13.1):*

<div style="border: 2px solid gray;font-size:80%; background-color: #F0F0F0; line-height: 1.5;">
<code>
iob_list = [
("ein", "DET", "B-NP"),
("Beispiel", "N", "I-NP"),
]

from nltk import conlltags2tree
tree = conlltags2tree(iob_list)
tree.pretty_print(unicodelines=True)
</code>
</div>